In [ ]:
import cv2
import numpy as np
import argparse
import win32api,win32gui,win32ui,win32con
import torch
import time
import pyautogui
import pydirectinput

In [ ]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s',channels = 3)
model.to('cuda')
model.conf = 0.7
# # Images
# img1 = Image.open('dataset/Snipaste_2022-01-05_23-09-44.jpg')  # PIL image
# imgs = [img1]  # batch of images
# t1 = time.time()
# # Inference
# results = model(imgs, size=640)  # includes NMS
# t2 = time.time()
# print('t2-t1',t2-t1)
# Results

# Norm

In [ ]:
window_width = 600
window_height = 600
half_window_width = int(window_width/2)
half_window_height = int(window_height/2)
#480
hwnd = 0  # 窗口的编号，0号表示当前活跃窗口
# 根据窗口句柄获取窗口的设备上下文DC（Divice Context）
hwndDC = win32gui.GetWindowDC(hwnd)
# 根据窗口的DC获取mfcDC
mfcDC = win32ui.CreateDCFromHandle(hwndDC)
# mfcDC创建可兼容的DC
saveDC = mfcDC.CreateCompatibleDC()
# 创建bigmap准备保存图片
saveBitMap = win32ui.CreateBitmap()
# 获取监控器信息
MoniterDev = win32api.EnumDisplayMonitors(None, None)
# w = MoniterDev[0][2][2]
# h = MoniterDev[0][2][3]
# 为bitmap开辟空间
# saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
saveBitMap.CreateCompatibleBitmap(mfcDC, window_width, window_height)
# 高度saveDC，将截图保存到saveBitmap中
saveDC.SelectObject(saveBitMap)
# MouseX, MouseY = pyautogui.position()
MouseX, MouseY = 1280, 800

while True:
    # 截取从左上角（1280-half_window_width, 800-half_window_height）长宽为（w，h）的图片
    # saveDC.BitBlt((0, 0), (w, h), mfcDC, (0, 0), win32con.SRCCOPY)
    saveDC.BitBlt((0,0), (window_width, window_height), mfcDC, (1280-half_window_width, 800-half_window_height), win32con.SRCCOPY)
    # saveBitMap.SaveBitmapFile(saveDC, 'filename')
    signedIntsArray = saveBitMap.GetBitmapBits(True)
    im_opencv = np.frombuffer(signedIntsArray, dtype = 'uint8')
    im_opencv.shape = (window_width, window_height, 4)
    imgs = cv2.cvtColor(im_opencv, cv2.COLOR_BGRA2RGB)
    # results = model(imgs, size=intel_window_size)
    results = model(imgs, size=600)
    pd = results.pandas().xyxy[0]  # img1 predictions (pandas)
    pd = pd[pd['class'] == 0]
    if pd.shape[0] == 0:
        continue
    pd['aim_x'] = 0.5*pd['xmin'] + 0.5*pd['xmax']
    pd['aim_y'] = 0.8*pd['ymin'] + 0.2*pd['ymax']
    pd['dis'] = abs(MouseX-pd['aim_x']) + abs(MouseY - pd['aim_y'])
    pd_min = pd.sort_values(by='dis', ascending=True).iloc[0]
    aim_x = pd_min['aim_x'] - half_window_width
    aim_y = pd_min['aim_y'] - half_window_height
    # pydirectinput.moveTo(int(aim_x/1.5)+1280, int(aim_y/1.5)+800,duration=0)
    # win32 move mouse
    win32api.SetCursorPos((int(aim_x/1.5)+1280, int(aim_y/1.4)+800))
    # win32api.keybd_event(0x11, 0, 0, 0)
    # win32api.mouse_event(win32con.MOUSEEVENTF_MOVE,int(aim_x/1.5),int(aim_y/1.5))
    # pydirectinput.moveTo(int(aim_x)+1280, int(aim_y)+800,duration=0)
    # pydirectinput.click()
    # pyautogui.click(clicks = 1,duration=0.03)

    # win32 click
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0, 0, 0)
    time.sleep(0.1)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0, 0, 0)
    # 最小枪口连续子弹时间
    time.sleep(0.1)
''''''

# 动量预测

In [ ]:
intel_window_size = 480
half_intel_window_size = int(intel_window_size/2)
# 截图
hwnd = 0  # 窗口的编号，0号表示当前活跃窗口
# 根据窗口句柄获取窗口的设备上下文DC（Divice Context）
hwndDC = win32gui.GetWindowDC(hwnd)
# 根据窗口的DC获取mfcDC
mfcDC = win32ui.CreateDCFromHandle(hwndDC)
# mfcDC创建可兼容的DC
saveDC = mfcDC.CreateCompatibleDC()
# 创建bigmap准备保存图片
saveBitMap = win32ui.CreateBitmap()
# 获取监控器信息
MoniterDev = win32api.EnumDisplayMonitors(None, None)
# w = MoniterDev[0][2][2]
# h = MoniterDev[0][2][3]
two_aims = [[None,None],[None,None]]
while True:
    t1 = time.time()
    MouseX, MouseY = pyautogui.position()
    # 为bitmap开辟空间
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    saveBitMap.CreateCompatibleBitmap(mfcDC, intel_window_size, intel_window_size)
    # 高度saveDC，将截图保存到saveBitmap中
    saveDC.SelectObject(saveBitMap)
    # 截取从左上角（0，0）长宽为（w，h）的图片
    # saveDC.BitBlt((0, 0), (w, h), mfcDC, (0, 0), win32con.SRCCOPY)
    saveDC.BitBlt((0,0), (intel_window_size,intel_window_size), mfcDC, (1280-half_intel_window_size, 800-half_intel_window_size), win32con.SRCCOPY)
    # saveBitMap.SaveBitmapFile(saveDC, 'filename')
    signedIntsArray = saveBitMap.GetBitmapBits(True)
    im_opencv = np.frombuffer(signedIntsArray, dtype = 'uint8')
    im_opencv.shape = (intel_window_size, intel_window_size, 4)
    imgs = cv2.cvtColor(im_opencv, cv2.COLOR_BGRA2RGB)
    
    t2 = time.time()
    print('图像准备时间：',t2-t1)

    results = model(imgs, size=intel_window_size)

    t3 = time.time()
    print('模型预测时间：',t3-t2)

    pd = results.pandas().xyxy[0]  # img1 predictions (pandas)
    pd = pd[pd['class'] == 0]
    if pd.shape[0] == 0:
        # 重新初始化
        two_aims = [[None,None],[None,None]]
        continue
    pd['aim_x'] = 0.48*pd['xmin'] + 0.52*pd['xmax']
    pd['aim_y'] = 0.9*pd['ymin'] + 0.1*pd['ymax']
    pd['dis'] = abs(MouseX-pd['aim_x']) + abs(MouseY - pd['aim_y'])
    pd_min = pd.sort_values(by='dis', ascending=True).iloc[0]
    aim_x = pd_min['aim_x'] - half_intel_window_size
    aim_y = pd_min['aim_y'] - half_intel_window_size
    two_aims[0],two_aims[1] = two_aims[1],[aim_x,aim_y]

    t4 = time.time()
    print('计算aim时间:',t4-t3)

    if two_aims[0] == [None,None]:
        pydirectinput.moveTo(int(aim_x/1.5)+1280, int(aim_y/1.5)+800,duration=0)
        pyautogui.click(clicks = 1,interval=0.02)
    else:
        aimx_1,aimy_1 = two_aims[0]
        aimx_2,aimy_2 = two_aims[1]
        distance = abs(aimx_1-aimx_2) + abs(aimy_1-aimy_2)
        if distance > 25:
            continue
            pydirectinput.moveTo(int(aim_x/1.5)+1280, int(aim_y/1.5)+800,duration=0)
            pyautogui.click(clicks = 1,interval=0.02)
        else:
            delta_x = aimx_2 - aimx_1
            delta_y = aimy_2 - aimy_1
            aim_x = aimx_2 + 6*delta_x
            aim_y = aimy_2 + 2*delta_y
            pydirectinput.moveTo(int(aim_x/1.5)+1280, int(aim_y/1.5)+800,duration=0)
            pyautogui.click(clicks = 1,interval=0.02)
    t5 = time.time()
    print('click时间',t5-t4)
    break
    # pydirectinput.moveTo(int(aim_x)+1280, int(aim_y)+800,duration=0)
    # pydirectinput.click()
    # pyautogui.click(clicks = 1,duration=0.03)
    # pydirectinput.click()

# win32快速移动鼠标

In [ ]:
while True:
    t1=time.time()
    win32api.mouse_event(win32con.MOUSEEVENTF_MOVE,-10,-10)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0, 0, 0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0, 0, 0)
    t2 = time.time()
    print(t2-t1)
    time.sleep(2)